In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 16

class Autoencoder(nn.Module):
    def __init__(self, input_size=128):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm2 = nn.BatchNorm2d(channels_base)
        self.batchnorm3 = nn.BatchNorm2d(channels_base * 2)
        
        self.leaky_relu1 = nn.LeakyReLU(0.1)
        self.leaky_relu2 = nn.LeakyReLU(0.1)
        self.leaky_relu3 = nn.LeakyReLU(0.1)
        self.leaky_relu4 = nn.LeakyReLU(0.1)
        self.avgpool1 = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
        self.avgpool2 = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
        self.upscale1 = nn.Upsample(scale_factor=scale_factor)
        self.upscale2 = nn.Upsample(scale_factor=scale_factor)

    def forward(self, input):
        # encoder
        x = self.leaky_relu1(self.batchnorm2(self.conv1(input)))
        x = self.avgpool1(x)
        x = self.leaky_relu2(self.batchnorm3(self.conv2(x)))
        x = self.avgpool2(x)
        
        # decoder
        x = self.leaky_relu3(self.batchnorm2(self.convtrans1(x)))
        x = self.upscale1(x)
        x = self.leaky_relu4(self.batchnorm1(self.convtrans2(x)))
        x = self.upscale2(self.convtrans3(x))

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             160
       BatchNorm2d-2           [-1, 16, 32, 32]              32
         LeakyReLU-3           [-1, 16, 32, 32]               0
         AvgPool2d-4           [-1, 16, 16, 16]               0
            Conv2d-5           [-1, 32, 16, 16]           4,640
       BatchNorm2d-6           [-1, 32, 16, 16]              64
         LeakyReLU-7           [-1, 32, 16, 16]               0
         AvgPool2d-8             [-1, 32, 8, 8]               0
   ConvTranspose2d-9             [-1, 16, 8, 8]           4,624
      BatchNorm2d-10             [-1, 16, 8, 8]              32
        LeakyReLU-11             [-1, 16, 8, 8]               0
         Upsample-12           [-1, 16, 16, 16]               0
  ConvTranspose2d-13            [-1, 8, 16, 16]           1,160
      BatchNorm2d-14            [-1, 8,

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00474782 (0.01315187), PSNR 23.23505592 (22.65866531), SSIM 0.61116838 (0.54852851)
Finished training epoch 0
Validate: MSE 0.00418631 (0.00390901), PSNR 23.78168297 (24.18689251), SSIM 0.67224586 (0.74796619)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00250054 (0.00329019), PSNR 26.01965904 (24.87230732), SSIM 0.73022270 (0.67846040)
Finished training epoch 1
Validate: MSE 0.00385808 (0.00356065), PSNR 24.13628960 (24.58017114), SSIM 0.69166648 (0.76337892)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00274525 (0.00301005), PSNR 25.61417198 (25.24149574), SSIM 0.72315836 (0.71521290)
Finished training epoch 2
Validate: MSE 0.00332467 (0.00315294), PSNR 24.78251266 (25.11871203), SSIM 0.69825983 (0.76862737)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00283002 (0.00289382), PSNR 25.48210716 (25.40730952), SSIM 0.75273597 (0.73493294)
Finished training epoch 3
Validate: MSE 0.00393018 (0.0

Epoch: 30, MSE 0.00217359 (0.00236825), PSNR 26.62821579 (26.27736558), SSIM 0.77506173 (0.77759367)
Finished training epoch 30
Validate: MSE 0.00324839 (0.00277901), PSNR 24.88332367 (25.63164445), SSIM 0.70553404 (0.77453771)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00266967 (0.00236836), PSNR 25.73542023 (26.27647395), SSIM 0.75488335 (0.77761537)
Finished training epoch 31
Validate: MSE 0.00311744 (0.00277560), PSNR 25.06201935 (25.63940316), SSIM 0.70739257 (0.77485660)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00224134 (0.00236811), PSNR 26.49491310 (26.27950406), SSIM 0.78014958 (0.77773207)
Finished training epoch 32
Validate: MSE 0.00359489 (0.00299577), PSNR 24.44314766 (25.31754676), SSIM 0.70605195 (0.77312413)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00222961 (0.00235073), PSNR 26.51771545 (26.31323893), SSIM 0.77037591 (0.77775548)
Finished training epoch 33
Validate: MSE 0.00323172 (0.00276456), PSNR 

Epoch: 60, MSE 0.00214149 (0.00231750), PSNR 26.69283867 (26.37160292), SSIM 0.78889203 (0.77830811)
Finished training epoch 60
Validate: MSE 0.00312269 (0.00285769), PSNR 25.05470467 (25.53125120), SSIM 0.70417041 (0.77257974)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00216845 (0.00231340), PSNR 26.63850594 (26.37913035), SSIM 0.79275227 (0.77824354)
Finished training epoch 61
Validate: MSE 0.00306842 (0.00293132), PSNR 25.13085365 (25.43317548), SSIM 0.70528960 (0.77206621)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00250481 (0.00231620), PSNR 26.01224518 (26.37765912), SSIM 0.75498915 (0.77827677)
Finished training epoch 62
Validate: MSE 0.00318084 (0.00279667), PSNR 24.97458649 (25.61415074), SSIM 0.70207125 (0.77247413)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00212350 (0.00230937), PSNR 26.72948074 (26.38985603), SSIM 0.78002810 (0.77814215)
Finished training epoch 63
Validate: MSE 0.00307558 (0.00285776), PSNR 

Epoch: 90, MSE 0.00210838 (0.00228711), PSNR 26.76050758 (26.43007241), SSIM 0.77974880 (0.77846178)
Finished training epoch 90
Validate: MSE 0.00326620 (0.00296188), PSNR 24.85956573 (25.37547130), SSIM 0.70374990 (0.77020879)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00236748 (0.00228300), PSNR 26.25713348 (26.43787081), SSIM 0.76584643 (0.77849713)
Finished training epoch 91
Validate: MSE 0.00355136 (0.00328058), PSNR 24.49605370 (24.94157931), SSIM 0.70184124 (0.76727003)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00232141 (0.00229391), PSNR 26.34248543 (26.41950922), SSIM 0.78596669 (0.77860166)
Finished training epoch 92
Validate: MSE 0.00445193 (0.00410274), PSNR 23.51451683 (23.95399256), SSIM 0.68490100 (0.75210348)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00328073 (0.00228548), PSNR 24.84029388 (26.43300262), SSIM 0.74206030 (0.77838716)
Finished training epoch 93
Validate: MSE 0.00318799 (0.00297096), PSNR 

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00354203 (0.00313557), PSNR 24.50747871 (25.12235259), SSIM 0.69654834 (0.76476971)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()